In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gujarat-biomass-production/Distance_Matrix.csv
/kaggle/input/gujarat-biomass-production/preds1.csv
/kaggle/input/gujarat-biomass-production/sample_submission.csv
/kaggle/input/gujarat-biomass-production/Biomass_History.csv
/kaggle/input/gujarat-biomass-production/preds2.csv
/kaggle/input/gujarat-biomass-production/shell_pred_v3.csv
/kaggle/input/gujarat-biomass-production/xgb_lor.csv


In [59]:
distance=pd.read_csv("/kaggle/input/gujarat-biomass-production/Distance_Matrix.csv",index_col=0)
biomass=pd.read_csv('/kaggle/input/gujarat-biomass-production/Biomass_History.csv')
samplesubmission=pd.read_csv('/kaggle/input/gujarat-biomass-production/sample_submission.csv')
pred1=pd.read_csv('/kaggle/input/gujarat-biomass-production/xgb_lor.csv',index_col=0)
# pred1=pd.read_csv('/kaggle/input/gujarat-biomass-production/shell_pred_v3.csv')
pred1.sum()

2018    305778.715806
2019    322104.219554
dtype: float64

In [4]:
def truncate(f, n):
    '''Truncates/pads a float f to n decimal places without rounding'''
    s = '%.12f' % f
    i, p, d = s.partition('.')
    return float('.'.join([i, (d+'0'*n)[:n]]))

In [100]:
biomass_pred=pred1
# val=(pred1['2018'].values+pred1['2019'].values)/2
val=pred1['2019'].values
# val=pred1['2018'].values
dist=distance.values.copy()
wt=val[:,np.newaxis]+val[np.newaxis,:]
for i in range(distance.shape[0]):
    wt[i][i]=0
    dist[i][i]=1
weight=wt/dist
weight[weight==np.inf]=1e9

/tmp/ipykernel_32/2226081128.py:10: RuntimeWarning:

divide by zero encountered in divide



In [6]:
%%time
class DSU:
    def __init__(self):
        self.p = {}
        self.count = 0
        self.biomass={}
        self.initvalue={}
        self.partialjoin=[]
    
    def add(self, x,val):
        if x in self.p: return
        self.p[x] = x
        self.count += 1
        self.biomass[x]=val
        self.initvalue[x]=val
    
    def find(self, x):
        if x != self.p[x]:
            self.p[x] = self.find(self.p[x])
        return self.p[x]
    
    def union(self, x, y):
        xp, yp = self.find(x), self.find(y)
        if xp == yp: return False
        self.p[xp] = yp
        self.biomass[yp]+=self.biomass[xp]
        self.count -= 1
        return True
    
    def partialunion(self,x,y,amount):
        cap=amount-self.biomass[y]
        biomassperdist=[]
        for node in range(2418):
            if self.find(node)==x:
                depo=y
                wt=1e9
                if dist[node][depo]!=0: 
                    wt=self.initvalue[node]/dist[node][depo]
                value=self.initvalue[node]
                biomassperdist.append([wt,value,node])
        biomassperdist=sorted(biomassperdist,key=lambda l:l[0], reverse=True)
        for index,node in enumerate(biomassperdist):
            if node[1]<=cap:
                self.biomass[node[2]]=self.initvalue[node[2]]
                self.biomass[y]+=node[1]
                self.p[node[2]]=y
                cap-=node[1]
            elif cap>=0.001:
                self.biomass[node[2]]=self.initvalue[node[2]]
                self.biomass[y]+=cap
                self.biomass[node[2]]-=cap
                self.initvalue[node[2]]-=cap
                biomassperdist[index][1]-=cap
                if cap>0:
                    self.partialjoin.append([cap,node[2],depo])
                cap=0
                break
        mask=[]
        value=[]
        for node in biomassperdist[index:]:
            mask.append(node[2])
            value.append(node[1])
#         print(np.sum(value))
        tempmatrix=distance.iloc[mask,mask].T.values
        tempcost=np.dot(tempmatrix,value)
        tempindex=np.argmin(tempcost)
        parent=mask[tempindex]
        self.p[parent]=parent
        self.biomass[parent]=self.initvalue[parent]
        for index,i in enumerate(mask):
            #problem parent adding its own value to make it 2 times
            if i!=parent:
                self.p[i] = parent
                self.biomass[i]=value[index]
                self.initvalue[i]=value[index]
                self.biomass[parent]+=value[index]
#         print('biomass:',parent,self.biomass[parent])
    
    def is_connected(self, x, y):
        if x in self.p and y in self.p:
            return self.find(x) == self.find(y)
        else:
            return False

CPU times: user 33 µs, sys: 3 µs, total: 36 µs
Wall time: 42.4 µs


In [102]:
%%time
edges=[]
for i in range(2418):
    for j in range(2418):
        if i!=j:
            edges.append([weight[i][j],i,j])
edges=sorted(edges,key=lambda l:l[0], reverse=True)
dsu=DSU()
for i in range(2418):
    dsu.add(i,val[i])
for index in range(len(edges)):
    edge=edges[index]
    value=edge[0]
    i=edge[1]
    j=edge[2]
    if dsu.biomass[i]<dsu.biomass[j] and dsu.p[i]==i and dsu.p[j]==j and dsu.biomass[j]<20000:
        if dsu.biomass[j]+dsu.biomass[i]<=20000:
            dsu.union(i,j)
        else:
            dsu.partialunion(i,j,20000)

CPU times: user 37.3 s, sys: 1.07 s, total: 38.4 s
Wall time: 37.8 s


In [103]:
cluster=[]
for i in range(2418):
    cluster.append(dsu.find(i))
total=0
for j in np.unique(cluster):
    value=[]
    for i in range(2418):
        if dsu.find(i)==j:
            value.append(dsu.initvalue[i])
    for i in dsu.partialjoin:
        if dsu.find(i[2])==j:
            value.append(i[0])
    total+=np.sum(value)
    print(j,np.sum(value))
print(total)

75 20000.0
348 20000.0
609 20000.0
824 20000.0
929 20000.0
1036 20000.0
1216 20000.0
1290 20000.0
1301 20000.0
1391 20000.0
1608 20000.0
1692 20000.0
1761 20000.0
1936 20000.0
1974 20000.0
2068 20000.0
2321 2104.219554213807
322104.2195542138


In [105]:
import plotly.express as px

clustermod=cluster
for index,value in enumerate(clustermod):
    if index in np.unique(clustermod):
        clustermod[index]=-1000
fig = px.scatter_mapbox(biomass, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Index", 
                        hover_data=["Index"],
                        color=clustermod,
                        zoom=5, 
                        height=400,
                        width=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [106]:
uniquecluster=np.unique(cluster)
noofcluster=len(uniquecluster)
clustermask=[]
clustermap={}
for i in range(noofcluster):
    clustermap[uniquecluster[i]]=i
    clustermask.append([])
for i in range(2418):
    value=clustermap[cluster[i]]
    clustermask[value].append(i)
    

depoloc=[]
for index,mask in enumerate(clustermask):
    tempmatrix=distance.iloc[mask,mask].T.values
    tempvalue=val[mask]
    tempcost=np.dot(tempmatrix, tempvalue)
    partialedge=[]
    for i in dsu.partialjoin:
        if dsu.find(i[2])==uniquecluster[index]:
            partialedge.append([i[0],i[1]])
    for index,i in enumerate(mask):
        cost=0
        for j in partialedge:
            cost+=(j[0]*dist[j[1]][i])
        tempcost[index]+=cost
    tempindex=np.argmin(tempcost)
    depoloc.append(mask[tempindex])

In [113]:
# depoloc.remove(2321)
# depoloc.remove(2361)
del depoloc[0]
# del depoloc[-1]
del clustermask[0]
# del clustermask[-1]

In [11]:
def knapsackmask(val,dist,depoloc):
        biomassperdist=[]
        for node in range(2418):
            stats=[]
            for depo in depoloc:
                wt=1e9
                if dist[node][depo]!=0: 
                    wt=val[node]/dist[node][depo]
                value=val[node]
                stats.append([wt,value,node,depo])
            stats=sorted(stats,key=lambda l:l[0], reverse=True)
            biomassperdist.append(stats)
        biomassperdist=sorted(biomassperdist,key=lambda l:l[0][0], reverse=True)
        knapsack={}
        capacity={}
        for depo in depoloc:
            capacity[depo]=20000
            knapsack[depo]=[0]*2418
        index=0
        while index<2418:
            stats=biomassperdist[index]
            node=stats[0]
            depo=node[3]
            if capacity[depo]>=node[1]:
                knapsack[depo][node[2]]=node[1]
                capacity[depo]-=node[1]
                index+=1
            else:
                totalcap=0
                for tempdepo in depoloc:
                    totalcap+=capacity[tempdepo]
                if totalcap<0.001:
                    break
                red=capacity[depo]
                knapsack[depo][node[2]]=capacity[depo]
                capacity[depo]=0
                del stats[0]
                if(len(stats)==0):
                    index+=1
                    continue
                for i in range(len(stats)):
                    stats[i][1]=(stats[i][1]-red)
                    stats[i][0]-=(red/dist[stats[i][2]][stats[i][3]])
                biomassperdist[index]=sorted(stats,key=lambda l:l[0], reverse=True)
                biomassperdist=biomassperdist[:index]+sorted(biomassperdist[index:],key=lambda l:l[0][0], reverse=True)
        return knapsack


def calcdepovalue(depoloc,knapsack):
    depovalue=[]
    for depo in depoloc:
        mask=knapsack[depo]
        cost=0
        for node,value in enumerate(mask):
            if(value>0):
                value=truncate(value,3)
                cost+=value
        depovalue.append(cost)
    return depovalue

In [115]:
knapsack=knapsackmask(val,dist,depoloc)
depovalue=calcdepovalue(depoloc,knapsack)
print(list(zip(depovalue,depoloc)))

[(19999.943, 346), (19999.948, 549), (19999.937999999987, 687), (19999.951, 868), (19999.968000000004, 1038), (19999.950999999997, 1341), (19999.957999999995, 1230), (19999.929, 1432), (19999.943999999996, 1450), (19999.964000000007, 1605), (19999.963, 1639), (19999.948999999997, 1815), (19999.953, 2049), (19999.928999999993, 2030), (19999.930999999982, 1905)]


In [121]:
import plotly.express as px

fig = px.scatter_mapbox(biomass.iloc[depoloc], 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Index", 
                        hover_data=["Index", "2010"],
                        zoom=5, 
                        height=400,
                        width=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [122]:
# xgb loc on 2019
refmask=[[687,1230,1432,1605,2049],
        [1450,1639,1815,1905,2030],
        [346,1341,549,868,1038]]

In [88]:
refmask=[[753,1292,1432,1880,2187],
        [1469,1691,1905,1974,1924],
        [434,265,1267,868,1091]]

In [69]:
refmask=[[102,443,609,881,985],
        [1325,1333,1905,1860,2172],
        [1232,1432,1718,1531,1402]]

In [68]:
def calcdepoloc(depoloc,refmask):
    depolocimprove=[]
    refmaskimprove=[]
    
    depotorefmap={}
    for index in range(len(refmask)):
        mask=refmask[index]
        refmaskimprove.append([])
        for i in mask:
            depotorefmap[i]=index
            
    depoprod={}
    for index,depo in enumerate(depoloc):
        depoprod[depo]=depovalue[index] 
    
    minicost=0
    for index,mask in enumerate(clustermask):
        depono=depoloc[index]
        refindex=depotorefmap[depono]
        refcluster=refmask[refindex]
        tempmatrix=distance.iloc[mask,mask].T.values
        tempvalue=val[mask]
        tempcost=np.dot(tempmatrix, tempvalue)
        partialedge=[]
        for i in dsu.partialjoin:
            if dsu.find(i[2])==uniquecluster[index]:
                partialedge.append([i[0],i[1]])
        for index,i in enumerate(mask):
            cost=0
            for j in partialedge:
                cost+=(j[0]*dist[j[1]][i])
            tempcost[index]+=cost
        for index,i in enumerate(mask):
            cost=0
            for neardepo in refcluster:
                cost+=depoprod[neardepo]*dist[neardepo][i]
            tempcost[index]+=cost
        tempindex=np.argmin(tempcost)
        minicost+=np.min(tempcost)
        depolocimprove.append(mask[tempindex])
        
    for index,depo in enumerate(depoloc):
        refindex=depotorefmap[depo]
        deponew=depolocimprove[index]
        refmaskimprove[refindex].append(deponew)
        
    return depolocimprove,refmaskimprove,minicost


def calcrefloc(depolocimproved,refmaskimproved):
    depoprod={}
    for index,depo in enumerate(depolocimprove):
        depoprod[depo]=depovalue[index] 
    
    refloc=[]
    minicost=0
    for index,mask in enumerate(refmaskimprove):
        tempmatrix=distance.iloc[mask,:].T.values
        tempvalue=[]
        for i in mask:
            tempvalue.append(depoprod[i])
        tempcost=np.dot(tempmatrix, tempvalue)
        tempindex=np.argmin(tempcost)
        minicost+=np.min(tempcost)

        refloc.append(tempindex)
    print(minicost)
    return refloc

In [123]:
def calcdepoloc(depoloc,refmask):
    depolocimprove=[]
    refmaskimprove=[]
    
    refloc=calcrefloc(depoloc,refmask)
    
    depotorefmap={}
    for index in range(len(refmask)):
        mask=refmask[index]
        refmaskimprove.append([])
        for i in mask:
            depotorefmap[i]=index
            
    depoprod={}
    for index,depo in enumerate(depoloc):
        depoprod[depo]=depovalue[index] 
    
    minicost=0
    for index,mask in enumerate(clustermask):
        depono=depoloc[index]
        ref=refloc[depotorefmap[depono]]
        tempmatrix=distance.iloc[mask,mask].T.values
        tempvalue=val[mask]
        tempcost=np.dot(tempmatrix, tempvalue)
        
        partialedge=[]
        for i in dsu.partialjoin:
            if dsu.find(i[2])==uniquecluster[index]:
                partialedge.append([i[0],i[1]])
        for index,i in enumerate(mask):
            cost=0
            for j in partialedge:
                cost+=(j[0]*dist[j[1]][i])
            tempcost[index]+=cost
    
        for index,i in enumerate(mask):
            cost=0
            tempcost[index]+=depoprod[depono]*dist[i][ref]
        tempindex=np.argmin(tempcost)
        minicost+=np.min(tempcost)
        depolocimprove.append(mask[tempindex])
        
    for index,depo in enumerate(depoloc):
        refindex=depotorefmap[depo]
        deponew=depolocimprove[index]
        refmaskimprove[refindex].append(deponew)
        
    return depolocimprove,refmaskimprove,minicost


def calcrefloc(depolocimproved,refmaskimproved):
    depoprod={}
    for index,depo in enumerate(depolocimprove):
        depoprod[depo]=depovalue[index] 
    
    refloc=[]
    minicost=0
    for index,mask in enumerate(refmaskimprove):
        tempmatrix=distance.iloc[mask,:].T.values
        tempvalue=[]
        for i in mask:
            tempvalue.append(depoprod[i])
        tempcost=np.dot(tempmatrix, tempvalue)
        tempindex=np.argmin(tempcost)
        minicost+=np.min(tempcost)

        refloc.append(tempindex)
    print(minicost)
    return refloc

In [124]:
depolocimprove=depoloc
refmaskimprove=refmask
for i in range(10):
    depolocimprove,refmaskimprove,minicost=calcdepoloc(depolocimprove,refmaskimprove)
    print(minicost,depolocimprove)

26280114.231309295
35346392.0035777 [553, 806, 1012, 805, 925, 997, 1357, 1360, 1455, 1605, 1640, 1757, 1828, 1698, 1395]
13362025.5344962
34104009.14679771 [553, 806, 1012, 805, 925, 997, 1292, 1360, 1455, 1359, 1640, 1757, 1828, 1698, 1395]
12326221.428865999
34104009.14679771 [553, 806, 1012, 805, 925, 997, 1292, 1360, 1455, 1359, 1640, 1757, 1828, 1698, 1395]
12326221.428865999
34104009.14679771 [553, 806, 1012, 805, 925, 997, 1292, 1360, 1455, 1359, 1640, 1757, 1828, 1698, 1395]
12326221.428865999
34104009.14679771 [553, 806, 1012, 805, 925, 997, 1292, 1360, 1455, 1359, 1640, 1757, 1828, 1698, 1395]
12326221.428865999
34104009.14679771 [553, 806, 1012, 805, 925, 997, 1292, 1360, 1455, 1359, 1640, 1757, 1828, 1698, 1395]
12326221.428865999
34104009.14679771 [553, 806, 1012, 805, 925, 997, 1292, 1360, 1455, 1359, 1640, 1757, 1828, 1698, 1395]
12326221.428865999
34104009.14679771 [553, 806, 1012, 805, 925, 997, 1292, 1360, 1455, 1359, 1640, 1757, 1828, 1698, 1395]
12326221.428865999


In [125]:
refloc=calcrefloc(depolocimprove,refmaskimprove)
refloc

12326221.428865999


[1359, 1640, 806]

In [126]:
knapsack18=knapsackmask(pred1['2018'].values,dist,depolocimprove)
knapsack19=knapsackmask(pred1['2019'].values,dist,depolocimprove)

In [127]:
total=[[],[]]
for depo in depolocimprove:
    total[0].append(knapsack18[depo])
for depo in depolocimprove:
    total[1].append(knapsack19[depo])
print((np.sum(total[0],axis=0)<=pred1['2018'].values).sum(),(np.sum(total[1],axis=0)<=pred1['2019'].values).sum())

2418 2418


In [51]:
pred1[np.sum(total[1],axis=0)>pred1['2019'].values]

,2018,2019
1884,216.131318,188.151113


In [52]:
np.sum(total[1],axis=0)[1884]

188.15111339537555

In [128]:
depoloc=depolocimprove
refmask=refmaskimprove
depomap={}
for index,depo in enumerate(depoloc):
    depomap[depo]=index

In [129]:
#######################loc_df
location=[]
for i in depoloc:
    location.append([20182019,'depot_location',i,np.nan,np.nan])
for i in refloc:
    location.append([20182019,'refinery_location',i,np.nan,np.nan])
loc_df=pd.DataFrame(location,columns=samplesubmission.columns)

######################biomass_forcast
biomass_forcast=samplesubmission[samplesubmission['data_type']=='biomass_forecast']
pred=np.concatenate((pred1['2018'],pred1['2019']))
biomass_forcast['value']=pred
biomass_forcast=biomass_forcast.reset_index().drop(columns=['index'])

/tmp/ipykernel_32/3070831411.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [55]:
# biomass_forcast.loc[4302,'value']=188.152
# biomass_forcast.loc[2418+1884,'value']=188.152

In [130]:
transport=[]
# depo
depovalue=[[],[]]
#2018
for depo in depoloc:
    mask=knapsack18[depo]
    cost=0
    for node,value in enumerate(mask):
        if(value>0):
#             value=int(value)
            value=truncate(value,3)
            transport.append([2018,'biomass_demand_supply',node,depo,value])
            cost+=value
    depovalue[0].append(cost)
#2019
for depo in depoloc:
    mask=knapsack19[depo]
    cost=0
    for node,value in enumerate(mask):
        if(value>0):
#             value=int(value)
            value=truncate(value,3)
            transport.append([2019,'biomass_demand_supply',node,depo,value])
            cost+=value
    depovalue[1].append(cost)

#ref
#2018
for index,mask in enumerate(refmask):
    for i in mask:
        value=depovalue[0][depomap[i]]
        source=i
        destination=refloc[index]
        transport.append([2018,'pellet_demand_supply',source,destination,value])
#2019
for index,mask in enumerate(refmask):
    for i in mask:
        value=depovalue[1][depomap[i]]
        source=i
        destination=refloc[index]
        transport.append([2019,'pellet_demand_supply',source,destination,value])
transport_df=pd.DataFrame(transport,columns=samplesubmission.columns)

In [131]:
submission=pd.concat([loc_df,biomass_forcast,transport_df],axis=0)
submission.to_csv('submission_with_xgblorpred_2019.csv',index=False)